In [2]:
import os
import shutil

print(os.getcwd())

/home/zhk/mytrain/dogs-vs-cats


In [75]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *

import h5py

In [35]:
#使用flow_from_directory 的时候该文件夹必须有一个子文件夹

test_generator = gen.flow_from_directory("test2",class_mode=None)

Found 12500 images belonging to 1 classes.


In [78]:
def readModel(MODEL,image_size,model_name,lambda_func=None):
    
    W,H = image_size[0],image_size[1]
    input_tensor = Input((H,W,3))
    x = input_tensor
    if lambda_func:
        x = Lambda(lambda_func)(x)
    
    model_base = MODEL(input_tensor=x,weights='imagenet',include_top = False)
    model = Model(model_base.input,GlobalAveragePooling2D()(model_base.output))
    
    gen = ImageDataGenerator()
    train_generator = gen.flow_from_directory("train2",image_size,shuffle=False,batch_size=16)
    test_generator = gen.flow_from_directory("test2",image_size,shuffle=False,batch_size=16,class_mode=None)
    batch_size=16
    train = model.predict_generator(train_generator)
    test = model.predict_generator(test_generator)
    
    with h5py.File("gap_%s.h5"%model_name) as h:
        h.create_dataset("train",data=train)
        h.create_dataset("test",data=test)
        h.create_dataset("label",data=train_generator.classes)
    

In [121]:
readModel(ResNet50, (224, 224),"ResNet50_v2",lambda_func=resnet50.preprocess_input)
readModel(VGG16, (224, 224),"VGG16_v2",lambda_func=vgg16.preprocess_input)


/home/zhk/anaconda3/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Found 29000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.
Found 29000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


In [ ]:
readModel(InceptionV3, (299, 299), "InceptionV3",lambda_func=inception_v3.preprocess_input)
readModel(Xception, (299, 299), "Xception",lambda_func=xception.preprocess_input)

In [136]:

import h5py
import numpy as np
from sklearn.utils import shuffle
np.random.seed(2017)

X_train = []
X_test = []

for filename in ["gap_ResNet50_v2.h5", "gap_Xception.h5", "gap_InceptionV3.h5"]:
    with h5py.File(filename, 'r') as h:
        X_train.append(np.array(h['train']))
        X_test.append(np.array(h['test']))
        y_train = np.array(h['label'])

X_train = np.concatenate(X_train, axis=1)
X_test = np.concatenate(X_test, axis=1)

X_train, y_train = shuffle(X_train, y_train)

In [137]:
len(X_train)

29000

In [138]:
from keras.models import *
from keras.layers import *

np.random.seed(2019)

input_tensor = Input(X_train.shape[1:])
x = Dropout(0.5)(input_tensor)
x = Dense(1,activation='sigmoid')(x)
model = Model(input_tensor,x)

model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [139]:
model.fit(X_train,y_train,batch_size=128,validation_split=0.2,epochs=50)

Train on 23200 samples, validate on 5800 samples
Epoch 1/50
23200/23200 [==============================] - 32s 1ms/step - loss: 0.2580 - acc: 0.9103 - val_loss: 0.2276 - val_acc: 0.9255
Epoch 2/50
23200/23200 [==============================] - 4s 191us/step - loss: 0.2350 - acc: 0.9199 - val_loss: 0.2398 - val_acc: 0.9257
Epoch 3/50
23200/23200 [==============================] - 5s 216us/step - loss: 0.2322 - acc: 0.9217 - val_loss: 0.2383 - val_acc: 0.9157
Epoch 4/50
23200/23200 [==============================] - 5s 219us/step - loss: 0.2283 - acc: 0.9212 - val_loss: 0.2251 - val_acc: 0.9252
Epoch 5/50
23200/23200 [==============================] - 5s 234us/step - loss: 0.2228 - acc: 0.9224 - val_loss: 0.2302 - val_acc: 0.9241
Epoch 6/50
23200/23200 [==============================] - 6s 245us/step - loss: 0.2256 - acc: 0.9219 - val_loss: 0.2286 - val_acc: 0.9247
Epoch 7/50
23200/23200 [==============================] - 6s 264us/step - loss: 0.2240 - acc: 0.9211 - val_loss: 0.2312 - va

In [140]:
model.save('mode.h5')

In [141]:


import pandas as pd
from keras.preprocessing.image import *

df = pd.read_csv("sampleSubmission.csv")

image_size = (224, 224)
gen = ImageDataGenerator()
test_generator = gen.flow_from_directory("test2", image_size, shuffle=False, 
                                         batch_size=16, class_mode=None)

for i, fname in enumerate(test_generator.filenames):
    index = int(fname[fname.rfind('/')+1:fname.rfind('.')])
    df.loc[index-1,'label']=y_pred[i]

df.to_csv('pred.csv', index=None)
df.head(10)


Found 12500 images belonging to 1 classes.


,id,label
0,1,0.995
1,2,0.995
2,3,0.995
3,4,0.995
4,5,0.005
5,6,0.005
6,7,0.005
7,8,0.005
8,9,0.005
9,10,0.005


array([[0.995],
       [0.005],
       [0.005],
       ...,
       [0.995],
       [0.005],
       [0.005]], dtype=float32)